In [23]:
import os
import xml.etree.ElementTree as ET

In [24]:
root_path = 'e:\\COMP\\COMP9444\\project\\dumpsite_data\\VOC2012'
train_path = os.path.join(root_path, "train")
test_path = os.path.join(root_path, "test")
file_Annotations = os.path.join(train_path, "Annotations")

In [25]:
object_class = []
for each_xml in os.listdir(file_Annotations):
    pic_xml = os.path.join(file_Annotations, each_xml)
    tree = ET.parse(pic_xml)
    root = tree.getroot()
    for object_elem in root.findall('object'):
        name_elem_value = object_elem.find('name').text
        object_class.append(name_elem_value)
print(set(object_class))


{'domestic garbage', 'agriculture forestry', 'construction waste', 'mining waste', 'disposed garbage', 'industry waste'}
